In [1]:
import json

from rdkit import Chem
import scipy.stats as stats

from src.dataset import load_cyp_data_split, load_herg_data_split, load_pampa_data_split, load_synthetic_data_split
from src.utils import get_data_partition_on_substructure_presence

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\kamil\miniconda3\envs\masters\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [2]:
train_cyp, valid_cyp, test_cyp = load_cyp_data_split()
train_herg, valid_herg, test_herg = load_herg_data_split()
train_pampa, valid_pampa, test_pampa = load_pampa_data_split()
train_synthetic, valid_synthetic, test_synthetic = load_synthetic_data_split()

test_datasets = {
    'cyp': test_cyp,
    'herg': test_herg,
    'pampa': test_pampa,
    'synthetic': test_synthetic
}

Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
[16:06:36] WARNING: not removing hydrogen atom without neighbors
[16:06:36] WARNING: not removing hydrogen atom without neighbors
Found local copy...
Loading...
Done!


In [3]:
grad_cam_smiles = json.load(open('results/smiles_grad_cam.json'))
saliency_map_smiles = json.load(open('results/smiles_saliency_map.json'))

In [4]:
grad_cam_results = {}
for dataset_name in ['cyp', 'herg', 'pampa', 'synthetic']:
    grad_cam_results[dataset_name] = []
    for i in range(len(grad_cam_smiles[dataset_name])):
        results = []
        for smiles in grad_cam_smiles[dataset_name][i]:
            has_substr, no_substr = get_data_partition_on_substructure_presence(test_datasets[dataset_name], smiles)
            has_substr_y = [int(float(mol.GetProp("y"))) for mol in has_substr]
            no_substr_y = [int(float(mol.GetProp("y"))) for mol in no_substr]
            chi_test_table = [
                [sum(has_substr_y), len(has_substr_y) - sum(has_substr_y)],
                [sum(no_substr_y), len(no_substr_y) - sum(no_substr_y)]
            ]
            try:
                results.append(stats.chi2_contingency(chi_test_table, correction=False))
            except ValueError:
                results.append(None)
        grad_cam_results[dataset_name].append(results)


saliency_map_results = {}
for dataset_name in ['cyp', 'herg', 'pampa', 'synthetic']:
    saliency_map_results[dataset_name] = []
    for i in range(len(saliency_map_smiles[dataset_name])):
        results = []
        for smiles in saliency_map_smiles[dataset_name][i]:
            has_substr, no_substr = get_data_partition_on_substructure_presence(test_datasets[dataset_name], smiles)
            has_substr_y = [int(float(mol.GetProp("y"))) for mol in has_substr]
            no_substr_y = [int(float(mol.GetProp("y"))) for mol in no_substr]
            chi_test_table = [
                [sum(has_substr_y), len(has_substr_y) - sum(has_substr_y)],
                [sum(no_substr_y), len(no_substr_y) - sum(no_substr_y)]
            ]
            try:
                results.append(stats.chi2_contingency(chi_test_table, correction=False))
            except ValueError:
                results.append(None)
        saliency_map_results[dataset_name].append(results)

In [5]:
def did_test_pass(result, threshold=0.05):
    return result.pvalue < threshold

grad_cam_results_pass = {}
for dataset_name in ['cyp', 'herg', 'pampa', 'synthetic']:
    grad_cam_results_pass[dataset_name] = []
    for i in range(len(grad_cam_results[dataset_name])):
        results = []
        for result in grad_cam_results[dataset_name][i]:
            if result is None:
                results.append(False)
            else:
                results.append(did_test_pass(result))
        grad_cam_results_pass[dataset_name].append(results)

saliency_map_results_pass = {}
for dataset_name in ['cyp', 'herg', 'pampa', 'synthetic']:
    saliency_map_results_pass[dataset_name] = []
    for i in range(len(saliency_map_results[dataset_name])):
        results = []
        for result in saliency_map_results[dataset_name][i]:
            if result is None:
                results.append(False)
            else:
                results.append(did_test_pass(result))
        saliency_map_results_pass[dataset_name].append(results)

In [6]:
def calculate_top_p_values(smiles, explanation_results):
    p_values = {}
    top_p_values = {}
    for dataset_name in ['cyp', 'herg', 'pampa', 'synthetic']:
        p_values[dataset_name] = []
        smiles_already_checked = []
        for i in range(len(explanation_results[dataset_name])):
            for j, result in enumerate(explanation_results[dataset_name][i]):
                current_smiles = smiles[dataset_name][i][j]
                component_size = i + 3
                if result is not None and current_smiles not in smiles_already_checked:
                    smiles_already_checked.append(current_smiles)
                    p_values[dataset_name].append((component_size, j, result.pvalue))
        p_values[dataset_name] = sorted(p_values[dataset_name], key=lambda x: x[2])
        top_p_values[dataset_name] = p_values[dataset_name][:3]
    return top_p_values

In [7]:
def find_top_results(smiles_path, top_p_values):
    smiles = json.load(open(smiles_path))

    top_mols = {}
    for dataset_name in ["cyp", "herg", "pampa", "synthetic"]:
        top_mols[dataset_name] = []
        for comp_size, j, _ in top_p_values[dataset_name]:
            i = comp_size - 3
            top_mols[dataset_name].append(Chem.MolFromSmiles(smiles[dataset_name][i][j]))

    average_targets = {}
    for dataset_name in ["cyp", "herg", "pampa", "synthetic"]:
        average_targets[dataset_name] = []
        for comp_size, j, _ in top_p_values[dataset_name]:
            i = comp_size - 3
            has_substr, no_substr = get_data_partition_on_substructure_presence(test_datasets[dataset_name], smiles[dataset_name][i][j])
            has_substr_y = [int(float(mol.GetProp("y"))) for mol in has_substr]
            no_substr_y = [int(float(mol.GetProp("y"))) for mol in no_substr]
            average_targets[dataset_name].append((sum(has_substr_y) / len(has_substr_y), sum(no_substr_y) / len(no_substr_y)))
    return average_targets

In [8]:
top_p_values_grad_cam = calculate_top_p_values(grad_cam_smiles, grad_cam_results)
average_targets_grad_cam = find_top_results('results/smiles_grad_cam.json', top_p_values_grad_cam)

top_p_values_saliency_map = calculate_top_p_values(saliency_map_smiles, saliency_map_results)
average_targets_saliency_map = find_top_results('results/smiles_saliency_map.json', top_p_values_saliency_map)

In [9]:
average_targets_grad_cam

{'cyp': [(0.4742943548387097, 0.15767634854771784),
  (0.59375, 0.3488499452354874),
  (0.49606815203145477, 0.2765957446808511)],
 'herg': [(0.8205128205128205, 0.14285714285714285),
  (0.8272727272727273, 0.3333333333333333),
  (0.8272727272727273, 0.3333333333333333)],
 'pampa': [(0.7314814814814815, 0.8862876254180602),
  (0.7088607594936709, 0.8780487804878049),
  (0.7559055118110236, 0.8857142857142857)],
 'synthetic': [(0.5, 0.9130434782608695),
  (0.5, 0.8958333333333334),
  (1.0, 0.8344827586206897)]}

In [10]:
average_targets_saliency_map

{'cyp': [(0.4742943548387097, 0.15767634854771784),
  (0.59375, 0.3488499452354874),
  (0.49606815203145477, 0.2765957446808511)],
 'herg': [(0.8205128205128205, 0.14285714285714285),
  (0.8272727272727273, 0.3333333333333333),
  (0.8272727272727273, 0.3333333333333333)],
 'pampa': [(0.7314814814814815, 0.8862876254180602),
  (0.7051282051282052, 0.878419452887538),
  (0.7088607594936709, 0.8780487804878049)],
 'synthetic': [(1.0, 0.0), (1.0, 0.5384615384615384), (1.0, 0.808)]}

In [11]:
top_p_values_saliency_map

{'cyp': [(3, 6, 9.15100277677312e-37),
  (4, 9, 2.479752058276682e-27),
  (3, 7, 5.7467558731475885e-27)],
 'herg': [(3, 4, 3.3918134766802585e-08),
  (3, 3, 1.7705562195108747e-06),
  (4, 1, 1.7705562195108747e-06)],
 'pampa': [(3, 4, 0.00013769770127582053),
  (3, 6, 0.0001422317676455562),
  (3, 7, 0.00018979605363326577)],
 'synthetic': [(3, 0, 2.0884875837625387e-45),
  (3, 1, 1.2474874357494747e-18),
  (6, 8, 5.2278657832674425e-05)]}

In [24]:
import numpy as np

import matplotlib.pyplot as plt

dataset_name_mapping = {
    'cyp': 'CYP3A4',
    'herg': 'hERG',
    'pampa': 'PAMPA',
    'synthetic': 'Synthetic'
}

method_name_mapping = {
    'grad_cam': 'GradCAM',
    'saliency_map': 'Saliency Map'
}

def save_plots_of_average_targets(average_targets, method_name):
    for dataset_name, values in average_targets.items():
        values = sorted(values, key=lambda x: max(x), reverse=True)
        has_substr_values = [v[0] for v in values]
        no_substr_values = [v[1] for v in values]
        
        indices = np.arange(len(values))
        
        plt.figure(figsize=(8, 6))
        plt.barh(indices, has_substr_values, height=0.4, color='green', label='Has Component', zorder=3)
        plt.barh(indices + 0.4, no_substr_values, height=0.4, color='red', label='No Component', zorder=3)
        plt.grid(True, axis='x', which="both", linestyle='--', alpha=0.7, zorder=0)
        
        # Add title with dataset name and method name
        plt.title(f"{dataset_name_mapping[dataset_name]} - {method_name_mapping[method_name]}", fontsize=22)
        
        plt.xlabel('Average Target Value', fontsize=20)
        plt.xticks(fontsize=20)
        plt.legend(fontsize=14, loc='lower right')
        plt.xlim(0, 1.5)
        plt.gca().invert_yaxis()
        plt.gca().yaxis.set_visible(False)

        plt.savefig(f"results/stat_test/{dataset_name}_top_avg_target_differences_{method_name}.png", bbox_inches="tight")
        plt.close()

save_plots_of_average_targets(average_targets_grad_cam, "grad_cam")
save_plots_of_average_targets(average_targets_saliency_map, "saliency_map")

In [13]:
from rdkit.Chem import Draw

def draw_molecules(smiles_list, mols_per_row=1):
    mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]
    img = Draw.MolsToGridImage(mols, molsPerRow=mols_per_row, subImgSize=(300, 300))
    return img

for dataset_name in ['cyp', 'herg', 'pampa', 'synthetic']:
    smiles_to_draw = []
    for comp_size, smiles_index, _ in top_p_values_grad_cam[dataset_name]:
        i = comp_size - 3
        smiles = grad_cam_smiles[dataset_name][i][smiles_index]
        smiles_to_draw.append(smiles)
    img = draw_molecules(smiles_to_draw)
    with open(f"results/stat_test/{dataset_name}_top_3_mols_drawn_grad_cam.png",'wb+') as img_file:
        img_file.write(img.data)

In [14]:
for dataset_name in ['cyp', 'herg', 'pampa', 'synthetic']:
    smiles_to_draw = []
    for comp_size, smiles_index, _ in top_p_values_saliency_map[dataset_name]:
        i = comp_size - 3
        smiles = saliency_map_smiles[dataset_name][i][smiles_index]
        smiles_to_draw.append(smiles)
    img = draw_molecules(smiles_to_draw)
    with open(f"results/stat_test/{dataset_name}_top_3_mols_drawn_saliency_map.png",'wb+') as img_file:
        img_file.write(img.data)

### Table results

In [21]:
for dataset_name in grad_cam_results_pass.keys():
    for i in range(len(grad_cam_results_pass[dataset_name])):
        print(f"{dataset_name}, {i + 3} component size: {sum(grad_cam_results_pass[dataset_name][i])}")

cyp, 3 component size: 9
cyp, 4 component size: 8
cyp, 5 component size: 7
cyp, 6 component size: 6
cyp, 7 component size: 7
cyp, 8 component size: 10
herg, 3 component size: 5
herg, 4 component size: 5
herg, 5 component size: 6
herg, 6 component size: 4
herg, 7 component size: 3
herg, 8 component size: 2
pampa, 3 component size: 2
pampa, 4 component size: 1
pampa, 5 component size: 1
pampa, 6 component size: 2
pampa, 7 component size: 2
pampa, 8 component size: 3
synthetic, 3 component size: 1
synthetic, 4 component size: 1
synthetic, 5 component size: 1
synthetic, 6 component size: 3
synthetic, 7 component size: 2
synthetic, 8 component size: 3


In [22]:
for dataset_name in saliency_map_results_pass.keys():
    for i in range(len(saliency_map_results_pass[dataset_name])):
        print(f"{dataset_name}, {i + 3} component size: {sum(saliency_map_results_pass[dataset_name][i])}")

cyp, 3 component size: 9
cyp, 4 component size: 7
cyp, 5 component size: 4
cyp, 6 component size: 4
cyp, 7 component size: 6
cyp, 8 component size: 9
herg, 3 component size: 6
herg, 4 component size: 5
herg, 5 component size: 5
herg, 6 component size: 6
herg, 7 component size: 6
herg, 8 component size: 5
pampa, 3 component size: 3
pampa, 4 component size: 3
pampa, 5 component size: 2
pampa, 6 component size: 1
pampa, 7 component size: 1
pampa, 8 component size: 0
synthetic, 3 component size: 4
synthetic, 4 component size: 5
synthetic, 5 component size: 5
synthetic, 6 component size: 7
synthetic, 7 component size: 5
synthetic, 8 component size: 3
